In [11]:
import gaia2
dir(gaia2)

['Analyzer',
 'AnalyzerFactory',
 'AnalyzerFactory_create',
 'AnalyzerFactory_instance',
 'AnalyzerFactory_keys',
 'AnalyzerFactory_swigregister',
 'Analyzer_swigregister',
 'AndFilter',
 'AndFilter_swigregister',
 'CoordVector',
 'CoordVector_swigregister',
 'DataSet',
 'DataSet_mergeFiles',
 'DataSet_points',
 'DataSet_swigregister',
 'DistanceFunction',
 'DistanceFunctionFactory',
 'DistanceFunctionFactory_create',
 'DistanceFunctionFactory_instance',
 'DistanceFunctionFactory_keys',
 'DistanceFunctionFactory_swigregister',
 'DistanceFunction_swigregister',
 'EnumType',
 'Filter',
 'FilterParser',
 'FilterParser_parse',
 'FilterParser_swigregister',
 'FilterWrapper',
 'FilterWrapper_swigregister',
 'Filter_opFromString',
 'Filter_parse',
 'Filter_swigregister',
 'FixedLength',
 'FixedLengthException',
 'FixedLengthException_swigregister',
 'FloatVector',
 'FloatVector_swigregister',
 'FrozenDataSet',
 'FrozenDataSet_checkValidFrozenDataSet',
 'FrozenDataSet_swigregister',
 'FrozenDi

In [12]:
# display info:
cvar.verbose = True

In [3]:
ds = DataSet()

In [14]:
p1 = Point()
p1.load('/home/gordon/Music/ABMI-509-3_20140325_000000.sig')
p1.setName('ABMI-509-3_20140325_000000')

In [15]:
p = ds.point('ABMI-509-3_20140325_000000')

Exception: Unable to find point with id 'ABMI-509-3_20140325_000000'

In [41]:
#input = raw_input

with open('/home/gordon/filelist.yaml','w') as out:
    #line1 = raw_input("ABMI-509-3_20140324_000000": "ABMI-509-3_20140324_000000.sig")
    #line2 = raw_input("ABMI-509-3_20140325_000000": "ABMI-509-3_20140325_000000.sig")
    line1 = raw_input("ABC")
    line2 = raw_input("DEF")
    #out.write('{}\n{}\n{}\n'.format(line1,line2))
    out.writelines([line1, "\n", line2,"\n"])
#ds = DataSet.mergeFiles(yaml.load(open('/home/gordon/filelist1.yaml').read())

ABC"ABMI-509-3_20140324_000000": "ABMI-509-3_20140324_000000.sig"
DEF"ABMI-509-3_20140325_000000": "ABMI-509-3_20140325_000000.sig"


In [43]:
ds = DataSet.mergeFiles(yaml.load(open('/home/gordon/filelist.yaml').read())

SyntaxError: unexpected EOF while parsing (<ipython-input-43-81447d6d4892>, line 1)

In [28]:
ds.save('mytrancemusic.db')
tranceDB = DataSet()
tranceDB.load('mytrancemusic.db')


In [29]:
p = ds.point('ABMI-509-3_20140325_000000')
# print some useful stuff
print 'BPM is:', p.value('tempotap_bpm')
print 'Variance of third MFCC coeff is:', p.value('mfcc.var')[2]
print 'Key/mode is: ', p.label('key_key'), p.label('key_mode')

# more pythonic way of doing it:
print 'BPM is:', p['tempotap_bpm']
print 'Key/mode is: ', p['key_key'], p['key_mode']

Exception: Unable to find point with id 'ABMI-509-3_20140325_000000'

In [47]:
import yaml


In [51]:
print yaml.dump(yaml.load('/home/gordon/filelist.yaml'))
for data in yaml.load_all('/home/gordon/filelist.yaml'):
    print data

/home/gordon/filelist.yaml
...

/home/gordon/filelist.yaml


In [46]:
ds.save('mytrancemusic.db')
tranceDB = DataSet()
tranceDB.load('mytrancemusic.db')

# from there it's just as before:
print 'How fast does a monkey dance:', tranceDB.point('Funky Monkey').value('tempotap_bpm')

How fast does a monkey dance:

Exception: Unable to find point with id 'Funky Monkey'

In [45]:
# remove MFCC, because we don't want them! (incidentally, mfcc also contains (as of essentia 0.4)
# covariance and inverse covariance matrix, which do NOT like to be normalized)
no_mfccDB = transform(cleanTranceDB, 'remove', { 'descriptorNames': '*mfcc*' })

# Normalize everything
normalizedDB = transform(no_mfccDB, 'normalize')

# PCA-30 on only the mean and variance of Real descriptors
pcaDB = transform(normalizedDB, 'pca', { 'dimension': 30, 'descriptorNames': [ '*.mean', '*.var' ] })

NameError: name 'cleanTranceDB' is not defined

In [44]:
euclideanMetric = DistanceFunctionFactory.create('euclidean', pcaDB.layout())
manhattanMetric = DistanceFunctionFactory.create('manhattan', pcaDB.layout())

# define a composite distance which is an euclidean distance on only the mean and var descriptors,
# plus 2 times the manhattan distance on the min and max descriptors:

euclidParams = {
    'name': 'euclidean',
    'params': { 'descriptorNames': [ '*.mean', '*.var' ] },
    'weight': 1.0
}

manhattanParams = {
    'name': 'manhattan',
    'params': { 'descriptorNames': [ '*.min', '*.max' ] },
    'weight': 2.0
}

mixedParams = { 'euclidp': euclidParams, 'manp': manhattanParams }

mixedMetric = DistanceFunctionFactory.create('linearcombination', pcaDB.layout(), mixedParams)


NameError: name 'pcaDB' is not defined

In [1]:
from __future__ import with_statement
import sys
from os.path import join, abspath
from optparse import OptionParser
from gaia2 import filedir
import gaia2.fastyaml as yaml

In [14]:
PROJECT_TEMPLATE = open('/home/gordon/gaia/scripts/classification/classification_project_template.yaml').read()

In [10]:
import os
import sys

#from setuptools import setup
#absFilePath = os.path.abspath(__file__)
#print(absFilePath)
#fileDir = os.path.dirname(os.path.abspath(__file__))
#print(fileDir)
print(os.getcwd())
print(sys.argv[0])
print(os.path.dirname(os.path.realpath('__file__')))

/home/gordon/gaia
/usr/lib/python2.7/dist-packages/ipykernel_launcher.py
/home/gordon/gaia


In [19]:
def generateProject(groundtruth_file, filelist_file, project_file, datasets_dir, results_dir):
    gt = yaml.load(open(groundtruth_file, 'r'))
    try:
        className = gt['className']
        groundTruth = gt['groundTruth']
    except:
        print groundtruth_file, "groundtruth file has incorrect format"
        sys.exit(2)

    fl = yaml.load(open(filelist_file, 'r'))

    gt_trackids = groundTruth.keys()
    fl_trackids = fl.keys()

    # check that there are no dublicate ids
    if len(gt_trackids) != len(set(gt_trackids)):
        print groundtruth_file, "contains dublicate track ids"
        sys.exit(3)

    if len(fl_trackids) != len(set(fl_trackids)):
        print filelist_file, "contains dublicate track ids"
        sys.exit(3)

    # check if filelist is consistent with groundtruth (no files missing)
    if set(gt_trackids) != set(fl_trackids):
        print "track ids found in", groundtruth_file, "are inconsistent with", filelist_file
        sys.exit(4)

    # write the project file
    with open(project_file, 'w') as pfile:
        pfile.write(PROJECT_TEMPLATE % { 'className': className,
                                         'datasetsDirectory': abspath(datasets_dir),
                                         'resultsDirectory': abspath(results_dir),
                                         'filelist': abspath(filelist_file),
                                         'groundtruth': abspath(groundtruth_file) })

    print 'Successfully written', project_file


#if __name__ == '__main__':
#    parser = OptionParser(usage = '%prog [options] groundtruth_file filelist_file project_file datasets_dir results_dir')#
#
#    options, args = parser.parse_args()
#
#    try:
#        groundtruth_file = args[0]
#        filelist_file = args[1]
#        project_file = args[2]
#        datasets_dir = args[3]
#        results_dir = args[4]
#    except:
#        parser.print_help()
#        sys.exit(1)
#
#    generateProject(groundtruth_file, filelist_file, project_file, datasets_dir, results_dir)



In [22]:
generateProject ('/home/gordon/gaia/scripts/classification/groundtruth_example.yaml',
                 '/home/gordon/gaia/scripts/classification/filelist_example.yaml',
                 '/home/gordon/gaia/scripts/classification/project_file.py',
                 '/home/gordon/gaia/scripts/classification',
                 '/home/gordon/gaia/scripts/classification')


Successfully written /home/gordon/gaia/scripts/classification/project_file.py
